In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from catboost import Pool, CatBoostClassifier

In [2]:
def read_and_preprocess(filepath):
    
    # initial reading
    data = pd.read_csv(filepath, index_col=0)
    
    # move species from index to the table
    data["species"] = data.index
    
    # create numeric idex
    data.index = range(len(data))
    
    # create categorical feature from Phylum (string)
    data["Phylum_Numeric"] = LabelEncoder().fit_transform(data["Phylum"].tolist())
    
    # remove features we will not use
    data = data.drop(["Phylum", "species", "occurrences", "NoSpecies"], axis=1)
    
    # create features and target dataframes
    features = data.drop(["extinct"], axis=1)
    target = data["extinct"]
    
    # create lists with categorical and continious features' names
    continious_cols = ["C_Cnumeric", "SC_Numeric"]
    categorical_cols = features.columns.drop(continious_cols).tolist()
    
    # make list of indexes
    categorical_idx = [features.columns.tolist().index(col) for col in categorical_cols]
        
    return features, target, categorical_idx

In [3]:
def model_me(filepath):
    
    # prepare data for modeling
    features, target, cat_idx = read_and_preprocess(filepath)
    
    # create cross-validation instance
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    
    # create holders for scores and feature importances
    cv_scores = []
    feature_importances = []
    
    # loop over different validation splits and save results
    for train_idx, test_idx in cv.split(features, target):
        
        # Gradient boosting model instance
        model = CatBoostClassifier(loss_function="Logloss", random_seed=0)
        
        # Create Pool data classes for train/test
        pool_train = Pool(features.iloc[train_idx, :], target[train_idx], cat_features=cat_idx)
        pool_test = Pool(features.iloc[test_idx, :], target[test_idx], cat_features=cat_idx)
        
        # Train model
        model.fit(pool_train, verbose=False)
        
        # save score from the individual split
        cv_scores.append(roc_auc_score(target[test_idx], model.predict_proba(pool_test)[:, 1]))
        
        # save feature importances from the individual split
        feature_importances.append(model.feature_importances_)
    
    cv_scores = pd.DataFrame(np.array(cv_scores),
                             index=["Split1", "Split2", "Split3", "Split4", "Split5"],
                             columns=["AUC"])
    
    feature_importances = pd.DataFrame(np.array(feature_importances), 
                                       index=["Split1", "Split2", "Split3", "Split4", "Split5"], 
                                       columns=features.columns)
    
    return cv_scores, feature_importances

In [4]:
%%time

# loop over individual Time Intervals
for i in range(1, 8):
    
    # obtain scores and feature importances using cross-validation
    cv, fi = model_me(f"../data/TimeInterval{i}.csv")
    
    cv.to_csv(f"../results/TimeInterval{i}_AUC.csv")
    fi.to_csv(f"../results/TimeInterval{i}_FI.csv")
    
    print(cv.mean())
    print(fi.mean())

AUC    0.748238
dtype: float64
K_Numeric          3.407116
Min_Numeric        6.441630
C_Numeric          4.450429
C_Cnumeric         4.743764
S_Numeric          4.029548
SC_Numeric         4.013417
O_Numeric          6.551575
T_Numeric          5.234228
M_Numeric          3.587277
R_Numeric          5.392170
Re_Numeric         4.589107
MinD_Numeric      17.089047
MaxD_Numeric       9.240485
System_Numeric    13.731161
Phylum_Numeric     7.499044
dtype: float64
AUC    0.834666
dtype: float64
K_Numeric          3.616735
Min_Numeric        4.501689
C_Numeric          5.981487
C_Cnumeric         6.703706
S_Numeric          5.757416
SC_Numeric         7.280582
O_Numeric          3.620392
T_Numeric          5.196225
M_Numeric          2.647740
R_Numeric          6.281273
Re_Numeric         3.051368
MinD_Numeric      14.138306
MaxD_Numeric      18.280224
System_Numeric    10.133928
Phylum_Numeric     2.808931
dtype: float64
AUC    0.80366
dtype: float64
K_Numeric          7.684355
Min_Numeri